<a href="https://colab.research.google.com/github/EmanueleGiavardi/AMD_project/blob/main/src/amd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q kaggle pyspark

In [4]:
import pyspark
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import os
from google.colab import files
from collections import Counter

In [5]:
# handling kaggle.json file

files.upload()
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
-rw-r--r-- 1 root root 72 May 22 09:28 kaggle.json


In [6]:
!kaggle datasets download -d "mohamedbakhet/amazon-books-reviews"
!unzip amazon-books-reviews.zip
!rm -r amazon-books-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 98% 1.04G/1.06G [00:06<00:00, 197MB/s]
100% 1.06G/1.06G [00:06<00:00, 177MB/s]
Archive:  amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_data.csv          


In [7]:
spark = pyspark.sql.SparkSession.builder.master("local[*]").appName("AMD_project").getOrCreate()
sc = spark.sparkContext

In [8]:
books_rating_df = spark.read.csv("Books_rating.csv", header=True, inferSchema=True)
books_rating_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)



In [58]:
# subsampling

random_state = 42
count = books_rating_df.count()
sampling_frac = 0.01

# probabilistic approach: keeps each line with prob = fraction
#books_rating_df_sub = books_rating_df.sample(fraction=sampling_frac, seed=random_state)

# keeps exactly (sampling_frac * count) lines, assuming books already in casual order
books_rating_df_sub = books_rating_df.limit(int(sampling_frac * count))
print(f"sample has {int(sampling_frac * count)} lines")

sample has 30000 lines


### **Exercise**
word occurrences in reviews summary

In [10]:
import re

def normalize_string(s):
    #s = re.sub('[^A-Za-z0-9]+', '', s)
    return s.lower().split(" ")

In [11]:
# map takes a function which take as input the ATOMIC INFORMATION of the rdd
# in this case: books_rating_df_sub.select("review/summary").rdd creates a new Spark Dataframe containing just the "review/summary" column and converts it into a rdd
# the atomic information of the rdd in this case is a Row, so our map function just extracts the "review/summary" field of the row, which is a string, and then tokenizes the string
# actually this is a flatMap, which means that if the input is a collection of other collections (in this case we have one list per row), the output is flatten: a unique list containing
# all the words of all the lists
s = books_rating_df_sub.select("review/summary").rdd.flatMap(lambda row: normalize_string(row["review/summary"]))

In [12]:
# here the atomic information of s is symply the word, thanks to the flatMap operation. So here we count the words, then we swap the (word, counter) pair so that we can
# orderByKey, and then we re-swap so that we can visualize the first 10 elements
s.map(lambda word:(word, 1)).reduceByKey(lambda x, y: x + y).map(lambda couple: (couple[1], couple[0])).sortByKey(ascending=False).map(lambda couple: (couple[1], couple[0])).take(10)

[('the', 6198),
 ('a', 6044),
 ('of', 4423),
 ('book', 3447),
 ('and', 2695),
 ('to', 2067),
 ('great', 2061),
 ('for', 1967),
 ('this', 1662),
 ('read', 1507)]

In [13]:
books_rating_df_sub.show()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

In [14]:
words = ["house", "dog", "cat", "dog", "garden", "cat", "dog",]

# create rdd

rdd = sc.parallelize(words)

rdd.map(lambda x:(x, 1)).reduceByKey(lambda x,y: x+y).collect()


[('dog', 3), ('house', 1), ('cat', 2), ('garden', 1)]

# **Link Analysis: finding influential/authoritative users**

**Graph**:
- nodes → users
- edges → links between users if two users reviewed the same book

the graph is **oriented**, so a link from ```u1``` to ```u2``` exists if ```u1``` and ```u2``` reviewd the same book, but the score (helpfulness) of the ```u1```'s review for that book is higher than the score that ```u2``` obtained for his/her review of that specific book.



## **Graph creation**

given $R$ the review table and given $R' = Π_{Title, User\_id, helpfulness}(R)$, we create the table

$$J = \sigma_{helpfulness_1 > helpfulness_2}(R' ⨝_{Title} R') $$

This table has the schema
```
root
 |-- User_id_1
 |-- Title
 |-- User_id_2
 |-- Helpfulness_1
 |-- Helpfulness_2
```
and it's build such that both ```User_id_1``` and ```User_id_2``` reviewd the book named with ```Title``` and ```Helpfulness_1``` $>$ ```Helpfulness_2```

Starting from this table we create the graph according to the criterium explained above

In [60]:
from pyspark.sql.functions import split, col, when

# TODO: replace with normalized version
def get_helpfulness_score(col_name):
    num = split(col(col_name), "/").getItem(0).cast("float")
    den = split(col(col_name), "/").getItem(1).cast("float")
    return when(den != 0, num / den).otherwise(0.0)

R_first = books_rating_df_sub.select(["Title", "User_id", "review/helpfulness"])
R1 = R_first.alias("R1")
R2 = R_first.alias("R2")

J = R1.join(R2, col("R1.Title") == col("R2.Title")) \
      .filter(col("R1.User_id") != col("R2.User_id")) \
      .select(
          col("R1.Title").alias("Title"),
          col("R1.User_id").alias("User_id_1"),
          col("R2.User_id").alias("User_id_2"),
          get_helpfulness_score("R1.review/helpfulness").alias("helpfulness_1"),
          get_helpfulness_score("R2.review/helpfulness").alias("helpfulness_2")
      )

J_filtered = J.filter(col("helpfulness_1") > col("helpfulness_2"))
J_filtered.count()

2735994

In [61]:
J_filtered.take(10)

[Row(Title='Dr. Seuss: American Icon', User_id_1='A30TK6U7DNS82R', User_id_2='A3VA4XFS5WNJO3', helpfulness_1=1.0, helpfulness_2=0.6),
 Row(Title='Dr. Seuss: American Icon', User_id_1='A30TK6U7DNS82R', User_id_2='A25MD5I2GUIW6W', helpfulness_1=1.0, helpfulness_2=0.0),
 Row(Title='Dr. Seuss: American Icon', User_id_1='A30TK6U7DNS82R', User_id_2='A2RSSXTDZDUSH4', helpfulness_1=1.0, helpfulness_2=0.0),
 Row(Title='Dr. Seuss: American Icon', User_id_1='A30TK6U7DNS82R', User_id_2='A14OJS0VWMOSWO', helpfulness_1=1.0, helpfulness_2=0.75),
 Row(Title='Dr. Seuss: American Icon', User_id_1='A30TK6U7DNS82R', User_id_2='A3UH4UZ4RSVO82', helpfulness_1=1.0, helpfulness_2=0.9090909090909091),
 Row(Title='Dr. Seuss: American Icon', User_id_1='A3UH4UZ4RSVO82', User_id_2='A3VA4XFS5WNJO3', helpfulness_1=0.9090909090909091, helpfulness_2=0.6),
 Row(Title='Dr. Seuss: American Icon', User_id_1='A3UH4UZ4RSVO82', User_id_2='A25MD5I2GUIW6W', helpfulness_1=0.9090909090909091, helpfulness_2=0.0),
 Row(Title='Dr. 

In [74]:
J_filtered.printSchema()

root
 |-- Title: string (nullable = true)
 |-- User_id_1: string (nullable = true)
 |-- User_id_2: string (nullable = true)
 |-- helpfulness_1: double (nullable = true)
 |-- helpfulness_2: double (nullable = true)



In [76]:
unique_users = J_filtered.select(col("User_id_1").alias("User_id")) \
    .union(J_filtered.select(col("User_id_2").alias("User_id"))) \
    .distinct()

user_ids_rdd = unique_users.rdd.map(lambda row: row["User_id"]).zipWithIndex()

In [78]:
user_ids_rdd.take(100)

[('AT3C9SZ3MB4U9', 0),
 ('A2CIIL55BUQWBG', 1),
 ('A140XH16IKR4B0', 2),
 ('A3EN6NDS6S7N9N', 3),
 ('AKDP4PZ94N2E1', 4),
 ('AN7O864X6GFGS', 5),
 ('A2BZRYYBIU3TPH', 6),
 ('A2V6MCBFM4BR9W', 7),
 ('A34IMKI2BF4936', 8),
 ('A2VG0RY3EUWZKB', 9),
 ('A2Y927PCQ6G556', 10),
 ('A1MN8ZRZA0A1ZP', 11),
 ('A2IYME501MHPC7', 12),
 ('A20MVVFHOAVC4U', 13),
 ('A2JKKRSTMLJ73H', 14),
 ('A2F1V4XDY8KTP6', 15),
 ('A3K7M5E4NNWA5L', 16),
 ('A4DF7XDJ5TL8V', 17),
 ('A35820ETLMB68', 18),
 ('A1SC29HW0HZAKX', 19),
 ('A24NQ8FKOAMF2Y', 20),
 ('A3CZ763XOAIQTV', 21),
 ('A1NBSKWJDCGB0S', 22),
 ('A2LISE436ZBCMB', 23),
 ('A3SHV3QBHVCKC6', 24),
 ('A2ZX1G5I2V4674', 25),
 ('A3LY4P70LKSVVA', 26),
 ('A36QI97W0DY09F', 27),
 ('AYBFMIUC6XBSG', 28),
 ('A1AJ2TKCLVKH7W', 29),
 ('A2EXCOHIQ5V7DZ', 30),
 ('A3U7ZU99ZOQO4', 31),
 ('ABB4NMEBHULVI', 32),
 ('A2759VIZDS0GBU', 33),
 ('A3MJWG85BEC3U5', 34),
 ('A12UB9OEVFO2Y3', 35),
 ('A20BBYODF7EFMS', 36),
 ('A356HHSZRK0LZ9', 37),
 ('A3RONGLCML5QQ3', 38),
 ('AGBRTPPIJT6UK', 39),
 ('ADY90IXWW6X2R', 